# Game 3 - Hospital Resources

Skillfully allocating medical resources during a pandemic can save lives. Unexpected and extreme medical events not only disrupt hospitals' normal routines and protocol, but strain resources such as trained staff, space and pharmaceuticals, in addition to PPE and life-saving medical equipment. Determining how to most effectively allocate these resources is paramount to assuring the wellbeing of Sailors far from port.

In this scenario, you will oversee the medical bay of the USS GHOST where COVID-19 has taken hold of the crew. Your task is to create an algorithm to deploy the ship's available resources in a manner that will ensure as many Sailors as possible recover from the virus.

## brainstorming:
- reinforcement learning?

In [219]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations

pd.set_option("display.max_rows", 100)

* +30 10 ventilators (reusable)
* +20 10 oxygen masks (reusable)
    * cannot be combined with ventilator
* +15 10 plasma
* +30 7 remdesivir
* +25 20 Dexamethasone
* +15 10 casirivimab
* +10 17 chloroquine
* 30 beds
    * bed boosts health 5 points
    * bed allocation required for other treatments
* final health score = final health + bed bonus + rate of decline + treatment bonuses
* penalty of -100 if any sailor final health is 0


In [233]:
sailors_df = pd.read_csv("day1data.csv", header=1, index_col=0)
sailors_df2 = pd.read_csv("day2data.csv", header=1, index_col=0)
sailors_df3 = pd.read_csv("day3data.csv", header=1, index_col=0)
# sailors_df.drop("Unnamed: 3", axis=1, inplace=True)
resource_list = ["Bed", "Ventilator", "Remdesivir", "Dexamethasone", "Plasma", "Casirivimab", "Supplemental oxygen", "Chloroquine"]
for resource in resource_list:
    sailors_df[resource] = 0
    sailors_df2[resource] = 0
    sailors_df3[resource] = 0
sailors_df["Used"] = ""
sailors_df2["Used"] = ""
sailors_df3["Used"] = ""
columns = ["resource", "count", "amt_used", "reusable", "health_bonus"]
data = np.array([["Bed", 30, 0, True, 5],
          ["Ventilator", 10, 0, True, 30],
          ["Supplemental oxygen", 10, 0, True, 20],
          ["Plasma", 10, 0, False, 15],
          ["Remdesivir", 7, 0, False, 30],
          ["Dexamethasone", 20, 0, False, 25],
          ["Casirivimab", 10, 0, False, 15],
          ["Chloroquine", 17, 0, False, 10]])
resources_df = pd.DataFrame(data=data, columns=columns)
for col in ["count", "amt_used", "health_bonus"]:
    resources_df[col] = pd.to_numeric(resources_df[col])
print(resources_df,"\n")

              resource  count  amt_used reusable  health_bonus
0                  Bed     30         0     True             5
1           Ventilator     10         0     True            30
2  Supplemental oxygen     10         0     True            20
3               Plasma     10         0    False            15
4           Remdesivir      7         0    False            30
5        Dexamethasone     20         0    False            25
6          Casirivimab     10         0    False            15
7          Chloroquine     17         0    False            10 



In [221]:
import random
sailors_df3 = pd.DataFrame()
for i in range(55):
    sailors_df3.at[i, "Health"] = random.randrange(1, 101)
    sailors_df3.at[i, "Rate of health decline"] = random.randrange(-99, 0)
resource_list = ["Bed", "Ventilator", "Remdesivir", "Dexamethasone", "Plasma", "Casirivimab", "Supplemental oxygen", "Chloroquine"]
for resource in resource_list:
    sailors_df3[resource] = 0
sailors_df3["Used"] = ""
sailors_df["Health"].mean(), sailors_df2["Health"].mean(), sailors_df3["Health"].mean()

(69.7, 63.77777777777778, 58.127272727272725)

In [234]:
sailors_df3_temp = sailors_df3.copy()
sailors_df3_temp["Total"] = sailors_df3["Health"]+sailors_df3["Rate of health decline"]
sailors_df3_temp.sort_values(by=["Total"], inplace=True)
sailors_df3_temp.head(100)

,Health,Rate of health decline,Unnamed: 3,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Used,Total
ID,,,,,,,,,,,,,
86,29,-99,NaN,0,0,0,0,0,0,0,0,,-70
77,24,-90,NaN,0,0,0,0,0,0,0,0,,-66
80,44,-95,NaN,0,0,0,0,0,0,0,0,,-51
79,48,-95,NaN,0,0,0,0,0,0,0,0,,-47
81,55,-98,NaN,0,0,0,0,0,0,0,0,,-43
85,57,-99,NaN,0,0,0,0,0,0,0,0,,-42
82,57,-99,NaN,0,0,0,0,0,0,0,0,,-42
83,58,-99,NaN,0,0,0,0,0,0,0,0,,-41
84,60,-99,NaN,0,0,0,0,0,0,0,0,,-39


In [235]:
def use_resource(sailors_df, resources_df, resource, indices):
    """
    resource = resource name as string
    indices = Pandas index list
    Ex. for single indices do [df.index[position]]
    Ex. for range of values do df.index[start:end]
    """
    bonus_dict = {}
    for r, bonus in zip(resources_df["resource"], resources_df["health_bonus"]):
        bonus_dict[r] = int(bonus)

    sailors_df.loc[indices, resource] = 1
    sailors_df.loc[indices, "Total"] += bonus_dict[resource]
    resources_df.loc[resources_df["resource"] == resource, "amt_used"] += len(indices)
    resources_df.loc[resources_df["resource"] == resource, "count"] -= len(indices)

def allocate_reusable(sailors_df, resources_df):    
    add_beds = sailors_df.index[:30]
    use_resource(sailors_df, resources_df, "Bed", add_beds)
    
    add_ventilators = sailors_df.index[:10]
    use_resource(sailors_df, resources_df, "Ventilator", add_ventilators)
    
    oxymask_candidates = sailors_df[(sailors_df["Ventilator"]==0) & (sailors_df["Bed"]>0)]
    add_oxymask = oxymask_candidates.index[:10]
    use_resource(sailors_df, resources_df, "Supplemental oxygen", add_oxymask)

def ded(sailors_df, resources_df):
    def get_bonus(tups):
        total = 0
        for name, bonus in tups:
            total += bonus
        return total
    def get_names(tups):
        names = []
        for name, bonus in tups:
            names.append(name)
        return names

    mindict = {}
    minlist = []
    dying = sailors_df.loc[(sailors_df["Total"] <= 0) & (sailors_df["Bed"] > 0)].sort_values(by=["Total"])
    for sailor in dying.itertuples():
        nonreusable = resources_df[(resources_df["reusable"]=="False") & (resources_df["count"] > 0)]
        tups = list(zip(nonreusable["resource"], nonreusable["health_bonus"]))
        y = sailor.Total
        comb_length = 1
        while len(minlist) == 0 and comb_length < len(tups):
            for comb in combinations(tups, comb_length):
                x = y + get_bonus(comb)
                mindict[x] = get_names(comb)
            minlist = [k for k in mindict.keys() if k > 0]
            comb_length += 1
        if len(minlist) == 0:
#             print("Sailor with Health:", sailors_df.iloc[2]["Health"], "is dying.")
            continue
        m = min(minlist)
        for resource in mindict[m]:
            use_resource(sailors_df, resources_df, resource, [sailor.Index])
        sailors_df.loc[sailor.Index, "Used"] = ','.join(mindict[m])
        mindict = {}
        minlist = []

def boosting(sailors_df, resources_df, threshold):
    if not threshold:
        return
    def get_bonus(tups):
        total = 0
        for name, bonus in tups:
            total += bonus
        return total
    def get_names(tups):
        names = []
        for name, bonus in tups:
            names.append(name)
        return names
    treatable = sailors_df.loc[(sailors_df["Bed"] > 0) & (sailors_df["Total"] > 0)]
    mindict = {}
    minlist = []
    for sailor in treatable.itertuples():
        unavailable = sailor.Used.split(',')
        available = resources_df[(~resources_df["resource"].isin(unavailable)) \
                                 & (resources_df["count"] > threshold) \
                                 & (resources_df["reusable"]=="False")]
        
        tups = list(zip(available["resource"], available["health_bonus"]))
        y = sailor.Total
        comb_length = len(tups)
        while len(minlist) == 0 and comb_length > 0:
            for comb in combinations(tups, comb_length):
                x = y + get_bonus(comb)
                mindict[x] = get_names(comb)
            minlist = [k for k in mindict.keys() if k < 101]
            comb_length -= 1
        if len(minlist) == 0:
            continue
        m = min(minlist)
        for resource in mindict[m]:
            
            use_resource(sailors_df, resources_df, resource, [sailor.Index])
        sailors_df.loc[sailor.Index, "Used"] = ','.join(mindict[m]+unavailable)
        mindict = {}
        minlist = []

def ethical_boosting(sailors_df, resources_df, boost_bool):
    if not boost_bool:
        return
    nonreusable = resources_df.loc[(resources_df["reusable"]=="False") & (resources_df["count"] > 0)].copy()
    sailor_idx = 0
    while nonreusable["count"].sum() > 0:
        sailors_df.sort_values(by=["Total"], inplace=True)
        unavailable = sailors_df.loc[sailors_df.index[sailor_idx], "Used"].split(',')
        nonreusable.sort_values(by=["health_bonus"], ascending=False, inplace=True)
        resource_idx = 0
        while resource_idx < len(nonreusable) and nonreusable.loc[nonreusable.index[resource_idx], "resource"] in unavailable:
            resource_idx += 1
        if resource_idx >= len(nonreusable) or sailors_df.loc[sailors_df.index[sailor_idx], "Bed"] == 0:
            sailor_idx += 1
        else:
            r = nonreusable.loc[nonreusable.index[resource_idx], "resource"]
            use_resource(sailors_df, resources_df, r, [sailors_df.index[sailor_idx]])
            sailors_df.loc[sailors_df.index[sailor_idx], "Used"] = ','.join([r]+unavailable)
            nonreusable = resources_df.loc[(resources_df["reusable"]=="False") & (resources_df["count"] > 0)].copy()

def allocation(sailors_df, resources_df, threshold):
    resources_df = resources_df.copy()
    sailors_df = sailors_df.copy()
    sailors_df["Total"] = sailors_df["Health"]+sailors_df["Rate of health decline"]
    sailors_df.sort_values(by=["Total"], inplace=True)
    
    allocate_reusable(sailors_df, resources_df)
    sailors_df["After Reusable"] = sailors_df["Total"]
    ded(sailors_df, resources_df)
    boosting(sailors_df, resources_df, threshold)
    
    return sailors_df, resources_df

def calc_final_score(allocation_df, resources_df):
    final_health = allocation_df["Total"].apply(lambda x: 100 if x > 100 else x)
    final_health = final_health.apply(lambda x: -100 if x < 1 else x)
    resources_df = resources_df[resources_df["reusable"]=="False"]
    #resource_bonus = (resources_df["count"]*resources_df["health_bonus"]).sum()/2
    #print(resource_bonus)
    return final_health.sum() #+ resource_bonus


In [238]:
a_df, r_df = allocation(sailors_df, resources_df, False)
x = calc_final_score(a_df, r_df)
a_df2, r_df2 = allocation(sailors_df2, r_df, 2)
y = calc_final_score(a_df2, r_df2)
a_df3, r_df3 = allocation(sailors_df3, r_df2, True)
z = calc_final_score(a_df3, r_df3)
print(x+y+z)
a_df3

3288


,Health,Rate of health decline,Unnamed: 3,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Used,Total,After Reusable
ID,,,,,,,,,,,,,,
86,29,-99,NaN,1,1,0,1,0,1,0,0,"Dexamethasone,Casirivimab",5,-35
77,24,-90,NaN,1,1,0,1,0,0,0,1,"Dexamethasone,Chloroquine",4,-31
80,44,-95,NaN,1,1,1,0,0,0,0,0,Remdesivir,14,-16
79,48,-95,NaN,1,1,0,0,0,1,0,0,Casirivimab,3,-12
81,55,-98,NaN,1,1,0,0,1,0,0,0,Plasma,7,-8
85,57,-99,NaN,1,1,0,0,1,0,0,0,Plasma,8,-7
82,57,-99,NaN,1,1,0,0,0,0,0,0,,-7,-7
83,58,-99,NaN,1,1,0,0,0,0,0,0,,-6,-6
84,60,-99,NaN,1,1,0,0,0,0,0,0,,-4,-4


In [237]:
a_df.drop(["Used", "After Reusable"], axis=1).sort_values(by="ID").to_csv("Challenge3_Day1Submission.csv")
a_df2.drop(["Unnamed: 3", "Used", "After Reusable"], axis=1).sort_values(by="ID").to_csv("Challenge3_Day2Submission.csv")
a_df3.drop(["Unnamed: 3", "Used", "After Reusable"], axis=1).sort_values(by="ID").to_csv("Challenge3_Day3Submission.csv")

In [ ]:
min_count = 5
for each sailor with Total above 0 with a bed:
    get list of resources that have not been applied to sailor and are above min_count
    get max combination of resources that can be applied to sailor that will not make Total > 100
        start with most resources and move backwards
    apply resources
    reset